In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
from skimage.transform import resize
from tensorflow.keras import datasets
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.optimizers import Adam
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from vae_vanilla import VAE

In [2]:
def load_data_for_model(batch_size=64):
    ## Loading dataset
    labels = np.load('./data/LFSD_labels.npy')
    depths = np.load('./data/LFSD_depths_repeated.npy')
    imgs = np.load('./data/LFSD_imgs.npy')
    masks = np.load('./data/LFSD_masks_single.npy')
    idx = np.random.permutation(len(labels))
    batch_idxs = [idx[i:i + batch_size] for i in range(0, len(labels), batch_size)]
    dataset = []
    for idx in batch_idxs:
        img_batch, depth_batch, mask_batch = imgs[idx], depths[idx], masks[idx]
        dataset.append((img_batch, depth_batch, mask_batch))
    train_dataset, test_dataset = train_test_split(dataset, test_size=0.3)
    print("Train dataset contains %d batches of %d samples each" % (len(train_dataset), batch_size))
    print("Test dataset contains %d batches of %d samples each" % (len(test_dataset), batch_size))
    return train_dataset, test_dataset

In [3]:
def merge_images(image_batch, size):
    h,w = image_batch.shape[1], image_batch.shape[2]
    img = np.zeros((int(h*size[0]), w*size[1]))
    for idx, im in enumerate(image_batch):
        im = np.squeeze(im, axis=2)
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = im
    return img

In [4]:
def train_round(train_dataset, test_dataset, learning_rate, model_name, epochs):
    latent_lookup = {
        'inception': 2048,
        'efficientnet': 1280,
        'vgg': 512,
        'mobilenet': 1280,
        'resnet': 2048,
    }
    latent_dim = latent_lookup[model_name]
    vae = VAE(latent_dim)
    vae.compile(optimizer=Adam(learning_rate))
    epochs = epochs
    # Train Step
    losses_across_epochs = {
        "loss": [],
        "reconstruction_loss": [],
        "kl_loss": [],
    }
    batch_num = len(train_dataset)
    for i in range(epochs):
        for k, v in losses_across_epochs.items():
            losses_across_epochs[k].append(0)
        for data in train_dataset:
            cur_loss = vae.train_step(data)
            for k, v in cur_loss.items():
                losses_across_epochs[k][-1] += cur_loss[k].numpy() / batch_num
            generated_image = vae.sample(data)
        print(f"Epoch {i} Total loss: { losses_across_epochs['loss'][-1]}")
        im_merged = merge_images(generated_image.numpy(), [8,8])
        plt.imsave('./images/vae_vanilla/%d.png' % i, im_merged, cmap='gray')
    for k, v in losses_across_epochs.items():
        
        np.save('./results/vae_vanilla/%s_%s' % (model_name, k), np.array(v))

    # Testing Step
    test_loss = 0
    for i, data in enumerate(test_dataset):
        _, _, mask_batch = data
        generated_image = vae.sample(data)
        reconstruction_loss = tf.reduce_sum(
            tf.keras.losses.binary_crossentropy(mask_batch, generated_image), [1,2]
        )
        test_loss += tf.reduce_mean(reconstruction_loss).numpy()
        im_merged = merge_images(generated_image.numpy(), [8,8])
        plt.imsave('./images/vae_vanilla/test_batch_%d.png' % i, im_merged, cmap='gray')
        
    test_loss = test_loss / len(test_dataset)
    np.save('./results/vae_vanilla/%s_test_loss' % model_name, np.array([test_loss]))
    return vae

In [5]:
def get_encoding_for_model(vae, model_name):

    train_dataset, test_dataset = None, None

    (train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()

    train_images = train_images / 255.0
    test_images = test_images / 255.0
    with tf.device('/cpu:0'):
        train_images = tf.image.resize(train_images, (256, 256)).numpy()
        test_images = tf.image.resize(test_images, (256, 256)).numpy()
    
    train_result, _, _ = vae.encode(train_images[:128], None, rand_depth=True)
    for i in tqdm(range(128, len(train_images), 128)):
        img = train_images[i:i+128]
        activation, _, _ = vae.encode(img, None, rand_depth=True)
        train_result = tf.concat((train_result, activation), axis=0)   
    np.save('./data/CIFAR100_vae_vanilla_%s_encoding_train.npy' % model_name, train_result.numpy())

    test_result, _, _ = vae.encode(test_images[:128], None, rand_depth=True)
    for i in tqdm(range(128, len(test_images), 128)):
        img = test_images[i:i+128]
        activation, _, _ = vae.encode(img, None, rand_depth=True)
        test_result = tf.concat((test_result, activation), axis=0)
    np.save('./data/CIFAR100_vae_vanilla_%s_encoding_test.npy' % model_name, test_result.numpy())

In [6]:
%%time
learning_rate = 1e-4
epochs = 100
for model_name in ['efficientnet', 'mobilenet','inception', 'resnet', 'vgg']:
    train_dataset, test_dataset = load_data_for_model()
    trained_model = train_round(train_dataset, test_dataset, learning_rate, model_name, epochs)
    print(f" {model_name} Gen encoding... ")
    get_encoding_for_model(trained_model, model_name)

Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2766788.194444445
Epoch 1 Total loss: 2653520.0555555555
Epoch 2 Total loss: 2561606.2222222225
Epoch 3 Total loss: 2441411.2777777775
Epoch 4 Total loss: 2322682.722222222
Epoch 5 Total loss: 2209099.3333333335
Epoch 6 Total loss: 2098112.1527777775
Epoch 7 Total loss: 1991821.5694444445
Epoch 8 Total loss: 1892266.1250000002
Epoch 9 Total loss: 1801601.5138888888
Epoch 10 Total loss: 1718186.3472222222
Epoch 11 Total loss: 1640825.625
Epoch 12 Total loss: 1569191.8194444447
Epoch 13 Total loss: 1503127.3750000002
Epoch 14 Total loss: 1442274.125
Epoch 15 Total loss: 1386370.013888889
Epoch 16 Total loss: 1335348.375
Epoch 17 Total loss: 1287831.5555555555
Epoch 18 Total loss: 1243395.4583333333
Epoch 19 Total loss: 1202111.8055555557
Epoch 20 Total loss: 1163554.1666666667
Epoch 21 Total loss: 1127631.4861111112
Epoch 22 Total loss: 1094076.0555555555
Epoch 23 To

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 48.16it/s]


Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2907912.305555556
Epoch 1 Total loss: 2898314.083333333
Epoch 2 Total loss: 2847795.555555556
Epoch 3 Total loss: 2749539.333333333
Epoch 4 Total loss: 2628281.7499999995
Epoch 5 Total loss: 2493439.777777778
Epoch 6 Total loss: 2351622.222222222
Epoch 7 Total loss: 2216191.6666666665
Epoch 8 Total loss: 2093844.902777778
Epoch 9 Total loss: 1983088.6111111112
Epoch 10 Total loss: 1882699.0416666667
Epoch 11 Total loss: 1791457.9444444445
Epoch 12 Total loss: 1708288.486111111
Epoch 13 Total loss: 1632231.7916666665
Epoch 14 Total loss: 1562538.736111111
Epoch 15 Total loss: 1498565.9722222222
Epoch 16 Total loss: 1439894.097222222
Epoch 17 Total loss: 1386263.0833333335
Epoch 18 Total loss: 1337417.2916666667
Epoch 19 Total loss: 1292605.6527777778
Epoch 20 Total loss: 1250892.4861111112
Epoch 21 Total loss: 1211813.1944444445
Epoch 22 Total loss: 1175155.48611111

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 72.53it/s]


Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2908722.25
Epoch 1 Total loss: 2902265.111111111
Epoch 2 Total loss: 2867498.6944444445
Epoch 3 Total loss: 2789949.972222222
Epoch 4 Total loss: 2682340.611111111
Epoch 5 Total loss: 2553250.583333333
Epoch 6 Total loss: 2405497.666666667
Epoch 7 Total loss: 2258688.111111111
Epoch 8 Total loss: 2123157.0416666665
Epoch 9 Total loss: 2001494.472222222
Epoch 10 Total loss: 1892416.513888889
Epoch 11 Total loss: 1794658.6666666667
Epoch 12 Total loss: 1706983.75
Epoch 13 Total loss: 1628001.888888889
Epoch 14 Total loss: 1556542.472222222
Epoch 15 Total loss: 1491775.4444444445
Epoch 16 Total loss: 1432465.6944444445
Epoch 17 Total loss: 1378113.6666666665
Epoch 18 Total loss: 1328409.9444444445
Epoch 19 Total loss: 1282826.2499999998
Epoch 20 Total loss: 1240684.9305555557
Epoch 21 Total loss: 1201839.5694444443
Epoch 22 Total loss: 1165893.9305555555
Epoch 23 Tota

...

Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2177125.8541666665
Epoch 1 Total loss: 2524114.027777778
Epoch 2 Total loss: 2443766.1111111115
Epoch 3 Total loss: 2314435.3333333335
Epoch 4 Total loss: 2189648.777777778
Epoch 5 Total loss: 2073485.6249999998
Epoch 6 Total loss: 1960253.6944444445
Epoch 7 Total loss: 1853744.736111111
Epoch 8 Total loss: 1755630.3472222225
Epoch 9 Total loss: 1665884.5555555555
Epoch 10 Total loss: 1584010.7083333333
Epoch 11 Total loss: 1508809.361111111
Epoch 12 Total loss: 1440031.597222222
Epoch 13 Total loss: 1377997.638888889
Epoch 14 Total loss: 1322138.4166666665
Epoch 15 Total loss: 1270897.4583333335
Epoch 16 Total loss: 1223515.3194444443
Epoch 17 Total loss: 1179907.375
Epoch 18 Total loss: 1139192.3194444445
Epoch 19 Total loss: 1101223.8194444443
Epoch 20 Total loss: 1065931.402777778
Epoch 21 Total loss: 1033017.673611111
Epoch 22 Total loss: 1002322.5972222222
Ep

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 71.44it/s]


Train dataset contains 9 batches of 64 samples each
Test dataset contains 5 batches of 64 samples each
Epoch 0 Total loss: 2907666.333333333
Epoch 1 Total loss: 2903294.027777778
Epoch 2 Total loss: 2873430.972222222
Epoch 3 Total loss: 2785701.4722222225
Epoch 4 Total loss: 2667162.277777778
Epoch 5 Total loss: 2545947.25
Epoch 6 Total loss: 2427074.888888889
Epoch 7 Total loss: 2308585.2222222225
Epoch 8 Total loss: 2196628.694444444
Epoch 9 Total loss: 2093401.236111111
Epoch 10 Total loss: 1998766.1666666665
Epoch 11 Total loss: 1911421.763888889
Epoch 12 Total loss: 1830741.208333333
Epoch 13 Total loss: 1755937.4027777778
Epoch 14 Total loss: 1686157.236111111
Epoch 15 Total loss: 1621085.0833333337
Epoch 16 Total loss: 1560487.8888888888
Epoch 17 Total loss: 1503894.5416666667
Epoch 18 Total loss: 1451234.9722222222
Epoch 19 Total loss: 1402086.6388888888
Epoch 20 Total loss: 1356044.1805555555
Epoch 21 Total loss: 1312958.1388888888
Epoch 22 Total loss: 1272566.3611111112
Epoch

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 75.87it/s]


Wall time: 13min 7s
